In [7]:
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
import gensim
import numpy as np
import pandas as pd

In [30]:
# Data
df_cleaned = pd.read_csv(r'tweets-data\cleaned.csv')

#slicing
df_cleaned = df_cleaned.iloc[:1001,:]
df_cleaned

,created_at,full_text,location,mentions,tokens,stemmed_tokens,clean_text
0,Tue Sep 17 23:55:49 +0000 2024,Yuk ganyang Fufufafa sesuai anjuran HRS PKS pa...,NaN,"['@UusRsd', '@msaid_didu']","['Yuk', 'ganyang', 'Fufufafa', 'sesuai', 'anju...","['yuk', 'ganyang', 'fufufafa', 'sesuai', 'anju...",yuk ganyang fufufafa sesuai anjur hrs pks bera...
1,Tue Sep 17 23:55:41 +0000 2024,Biasadrama mulyonopura kaget fufufafa fufufafaaaa,NaN,['@kompascom'],"['Biasadrama', 'mulyonopura', 'kaget', 'fufufa...","['biasadrama', 'mulyonopura', 'kaget', 'fufufa...",biasadrama mulyonopura kaget fufufafa fufufafaaaa
2,Tue Sep 17 23:55:11 +0000 2024,Jamet menjijikkan Mesum pula,"Jakarta Capital Region, Indone",['@fufu__fafa'],"['Jamet', 'menjijikkan', 'Mesum']","['jamet', 'jijik', 'mesum']",jamet jijik mesum
3,Tue Sep 17 23:55:00 +0000 2024,Nebeng temenato nebeng sopir temen Fufufafa fu...,NaN,['@satriohendri'],"['Nebeng', 'temenato', 'nebeng', 'sopir', 'tem...","['nebeng', 'temenato', 'nebeng', 'sopir', 'tem...",nebeng temenato nebeng sopir temen fufufafa fu...
4,Tue Sep 17 23:54:58 +0000 2024,kasian org ini model penakut Sama fufufafa yg ...,NaN,['@andikamalreza'],"['kasian', 'org', 'model', 'penakut', 'fufufaf...","['kasi', 'org', 'model', 'takut', 'fufufafa', ...",kasi org model takut fufufafa yg udah hina dia...
...,...,...,...,...,...,...,...
996,Tue Sep 17 11:19:02 +0000 2024,Fufufafa wkkk,NaN,"['@aewin86', '@BankSinarmas']","['Fufufafa', 'wkkk']","['fufufafa', 'wkkk']",fufufafa wkkk
997,Tue Sep 17 11:18:40 +0000 2024,Diem lu mending cebokin dulu sonoh si fufufafa,NaN,"['@Ayah3bidadari', '@AndiSinulingga']","['Diem', 'lu', 'mending', 'cebokin', 'sonoh', ...","['diem', 'lu', 'mending', 'cebokin', 'sonoh', ...",diem lu mending cebokin sonoh si fufufafa
998,Tue Sep 17 11:18:26 +0000 2024,Mungkin ada baiknya kita tunggu kapan menyanyi...,NaN,['@PKSejahtera_ID'],"['baiknya', 'tunggu', 'menyanyikan', 'lagu', '...","['baik', 'tunggu', 'nyanyi', 'lagu', 'ganyang'...",baik tunggu nyanyi lagu ganyang fufufafa dinya...
999,Tue Sep 17 11:18:10 +0000 2024,Mungkin ada baiknya kita tunggu kapan menyanyi...,NaN,"['@msaid_didu', '@PKSejahtera_ID']","['baiknya', 'tunggu', 'menyanyikan', 'lagu', '...","['baik', 'tunggu', 'nyanyi', 'lagu', 'ganyang'...",baik tunggu nyanyi lagu ganyang fufufafa dinya...


In [31]:
# Step 1: Preprocessing Data
# Mengganti NaN dengan string kosong
df_cleaned['clean_text'] = df_cleaned['clean_text'].fillna("")

In [32]:
# Step 2: Inisialisasi Model BERTopic
topic_model = BERTopic(language="indonesian", calculate_probabilities=True, verbose=True)

# Step 3: Latih Model pada Data
texts = df_cleaned['clean_text'].tolist()
topics, probs = topic_model.fit_transform(texts)

2024-10-01 13:51:06,355 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 32/32 [00:08<00:00,  3.85it/s]
2024-10-01 13:51:18,815 - BERTopic - Embedding - Completed ✓
2024-10-01 13:51:18,816 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-01 13:51:22,040 - BERTopic - Dimensionality - Completed ✓
2024-10-01 13:51:22,041 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-01 13:51:22,118 - BERTopic - Cluster - Completed ✓
2024-10-01 13:51:22,121 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-01 13:51:22,149 - BERTopic - Representation - Completed ✓


In [33]:
# Step 4: Mendapatkan Topik
topic_info = topic_model.get_topic_info()
topics_keywords = [topic_model.get_topic(topic) for topic in set(topics) if topic != -1]  # Mengabaikan topik -1 (outliers)

# Step 5: Mengatur Topik untuk Evaluasi Coherence
top_words_per_topic = []
for topic in topics_keywords:
    if topic:  # Memeriksa apakah topik ada
        top_words = [word for word, _ in topic[:10]]  # Ambil 10 kata teratas dari topik
        top_words_per_topic.append(top_words)

# Membuat representasi teks dalam bentuk list of list kata untuk gensim
texts_for_gensim  = [text.split() for text in df_cleaned['clean_text']]

# Membuat dictionary dan corpus
dictionary = gensim.corpora.Dictionary(texts_for_gensim)

# Menghitung Coherence Score dengan Gensim
coherence_model = CoherenceModel(topics=top_words_per_topic, texts=texts_for_gensim, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

In [40]:
# Menggunakan representasi vektor dari dokumen yang dihasilkan oleh BERTopic
embeddings, topics = topic_model.transform(texts)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches: 100%|██████████| 32/32 [00:07<00:00,  4.15it/s]
2024-10-01 14:07:00,435 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2024-10-01 14:07:00,438 - BERTopic - Dimensionality - Completed ✓
2024-10-01 14:07:00,439 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-10-01 14:07:00,471 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2024-10-01 14:07:00,562 - BERTopic - Probabilities - Completed ✓
2024-10-01 14:07:00,563 - BERTopic - Cluster - Completed ✓


In [45]:
embeddings = np.array(embeddings)
print("Shape of embeddings:", embeddings.shape)
print("Length of topics:", len(topics))

Shape of embeddings: (1001,)
Length of topics: 1001


In [46]:
# Menghitung Silhouette Score untuk clustering
silhouette_avg = silhouette_score(embeddings, topics)
print(f"Silhouette Score untuk BERTopic: {silhouette_avg}")

# Menampilkan hasil evaluasi
print(f"\nCoherence Score untuk BERTopic: {coherence_score}")

Silhouette Score tidak dapat dihitung, hanya ada satu topik atau noise.

Coherence Score untuk BERTopic: 0.5652129379833577


In [35]:
# Step 7: Menampilkan Topik yang Ditemukan dengan Format Lebih Baik
topic_info = topic_model.get_topic_info()
print("\nInformasi Topik yang Ditemukan:")

# Format penampilan topik
for index, row in topic_info.iterrows():
    print(f"\nTopik #{row['Topic']}:")
    print(f"  - Jumlah Dokumen  : {row['Count']}")
    print(f"  - Nama Topik      : {row['Name']}")
    print(f"  - Representasi    : {', '.join(row['Representation'][:5])}...")
    print(f"  - Dokumen Contoh  : {row['Representative_Docs'][:3]}")


Informasi Topik yang Ditemukan:

Topik #-1:
  - Jumlah Dokumen  : 39
  - Nama Topik      : -1_slot_gacor_keluarga_solo
  - Representasi    : slot, gacor, keluarga, solo, fufufafa...
  - Dokumen Contoh  : ['slot gacor new member wd link gacor nahan fufufafa arhan malam puncak paes slot gacor slotgacorhariini slotgacor nabrak ahmad dhani msbrew data pribadi indodax selasa kaesang gibran m labubu slot', 'slot gacor new member wd link gacor nahan fufufafa arhan malam puncak paes slot gacor slotgacorhariini slotgacor nabrak ahmad dhani msbrew data pribadi indodax selasa kaesang gibran m labubu slot', 'slot gacor new member wd link gacor nahan fufufafa arhan malam puncak paes slot gacor slotgacorhariini slotgacor nabrak ahmad dhani msbrew data pribadi indodax selasa kaesang gibran m labubu slot']

Topik #0:
  - Jumlah Dokumen  : 699
  - Nama Topik      : 0_fufufafa_yg_akun_aja
  - Representasi    : fufufafa, yg, akun, aja, ya...
  - Dokumen Contoh  : ['fufufafa', 'fufufafa', 'fufufafa']

To

In [36]:
# Step 7: Visualisasi Hierarki Topik
topic_model.visualize_hierarchy()

In [37]:
# Step 8: Visualisasi Distribusi Topik dalam Dokumen
topic_model.visualize_barchart()